In [1]:
from collections import Counter
from itertools import product
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch_model_base import TorchModelBase
from torch_rnn_classifier import TorchRNNClassifier, TorchRNNClassifierModel
from torch_rnn_classifier import TorchRNNClassifier
import nli
import os
import utils
import pickle

In [2]:
GLOVE_HOME = "/home/brett_szalapski/"

DATA_HOME = "/Users/bszalapski/Documents/StanfordCourses/CS224U/cs224u_project/cs224uSNLI/data"

SNLI_HOME = os.path.join(DATA_HOME, "nlidata/snli_1.0")

MULTINLI_HOME = os.path.join(DATA_HOME, "multinli_1.0")

ANNOTATIONS_HOME = os.path.join(DATA_HOME, "multinli_1.0_annotations")

In [3]:
train_reader = nli.SNLITrainReader(
    SNLI_HOME, samp_percentage=0.10, random_state=42)

In [4]:
def fit_softmax_classifier_with_preselected_params(X, y):       
    mod = LogisticRegression(
        fit_intercept=True, 
        penalty='l1', 
        solver='saga',  ## Required for penalty='ll'.
        multi_class='ovr',
        C=0.4)
    mod.fit(X, y)
    return mod

In [5]:
# Prepare SRL and glove embedding lookups
# from allennlp.predictors.predictor import Predictor
# predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/srl-model-2018.05.25.tar.gz")

srl_lookup = {}

glove_dim = 50
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.50d.txt'))
glove_lookup["$UNK"] = utils.randvec(n=glove_dim)

In [6]:
def glove_leaves_phi(t1, t2, np_func=np.concatenate):
# def glove_leaves_phi(ex, np_func=np.concatenate):
    """Represent `tree` as a combination of the vector of its words.
    
    Parameters
    ----------
    t1 : nltk.Tree   
    t2 : nltk.Tree   
    np_func : function (default: np.sum)
        A numpy matrix operation that can be applied columnwise, 
        like `np.mean`, `np.sum`, or `np.prod`. The requirement is that 
        the function take `axis=0` as one of its arguments (to ensure
        columnwise combination) and that it return a vector of a 
        fixed length, no matter what the size of the tree is.
    
    Returns
    -------
    np.array
            
    """
    if hasattr(t1, "leaves"):
        s1 = t1.leaves()
        s2 = t2.leaves()
    else:
        s1 = t1
        s2 = t2
    tags = predictor.predict_batch_json([{"sentence": " ".join(s1)}, {"sentence": " ".join(s2)}])
    
    prem_tags = _get_best_tags(tags[0], np_func)
    hyp_tags = _get_best_tags(tags[1], np_func)
#     print(f"{ex.sentence1}, {ex.sentence2}")
#     prem_tags = _embed_tags(ex.tags1, np_func)
#     hyp_tags = _embed_tags(ex.tags2, np_func)
    prem_tags_tens = torch.tensor(prem_tags, requires_grad=True)
    hyp_tags_tens = torch.tensor(hyp_tags, requires_grad=True)
#     print(prem_tags_tens.size())
    
    prem_words = _get_tree_vecs(t1, glove_lookup, np_func)
    hyp_words = _get_tree_vecs(t2, glove_lookup, np_func)
    prem_words_tens = torch.tensor(prem_words, requires_grad=False)
#     print(prem_words_tens.size())
    hyp_words_tens = torch.tensor(hyp_words, requires_grad=False)
    
    prem_vecs = torch.cat((prem_words_tens, prem_tags_tens))
    hyp_vecs = torch.cat((hyp_words_tens, hyp_tags_tens))
    
    return (prem_vecs, hyp_vecs)

In [7]:
class TorchRNNSentenceEncoderDataset(torch.utils.data.Dataset):
    def __init__(self, sequences, seq_lengths, y):
        self.prem_seqs, self.hyp_seqs = sequences
        self.prem_lengths, self.hyp_lengths = seq_lengths
        self.y = y
        assert len(self.prem_seqs) == len(self.y)

    @staticmethod
    def collate_fn(batch):
        X_prem, X_hyp, prem_lengths, hyp_lengths, y = zip(*batch)
        prem_lengths = torch.LongTensor(prem_lengths)
        hyp_lengths = torch.LongTensor(hyp_lengths)
        y = torch.LongTensor(y)
        return (X_prem, X_hyp), (prem_lengths, hyp_lengths), y

    def __len__(self):
        return len(self.prem_seqs)

    def __getitem__(self, idx):
        return (self.prem_seqs[idx], self.hyp_seqs[idx],
                self.prem_lengths[idx], self.hyp_lengths[idx],
                self.y[idx])

In [8]:
class TorchRNNSentenceEncoderClassifierModel(TorchRNNClassifierModel):
    def __init__(self, vocab_size, embed_dim, embedding, use_embedding,
            hidden_dim, output_dim, bidirectional, device):
        super(TorchRNNSentenceEncoderClassifierModel, self).__init__(
            vocab_size, embed_dim, embedding, use_embedding,
            hidden_dim, output_dim, bidirectional, device)
        self.hypothesis_rnn = nn.LSTM(
            input_size=2 * glove_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=self.bidirectional)
        if bidirectional:
            classifier_dim = hidden_dim * 2 * 2
        else:
            classifier_dim = hidden_dim * 2
        self.classifier_layer = nn.Linear(
            classifier_dim, output_dim)

    def forward(self, X, seq_lengths):
        X_prem, X_hyp = X
        prem_lengths, hyp_lengths = seq_lengths
        
        prem_state = self.rnn_forward(X_prem, prem_lengths, self.rnn)
        hyp_state = self.rnn_forward(X_hyp, hyp_lengths, self.hypothesis_rnn)
        state = torch.cat((prem_state, hyp_state), dim=1)
        logits = self.classifier_layer(state)
        return logits

In [9]:
class TorchRNNSentenceEncoderClassifier(TorchRNNClassifier):

    def build_dataset(self, X, y):
        X_prem, X_hyp = zip(*X)
        X_prem, prem_lengths = self._prepare_dataset(X_prem)
        X_hyp, hyp_lengths = self._prepare_dataset(X_hyp)
        return TorchRNNSentenceEncoderDataset(
            (X_prem, X_hyp), (prem_lengths, hyp_lengths), y)

    def build_graph(self):
        return TorchRNNSentenceEncoderClassifierModel(
            len(self.vocab),
            embedding=self.embedding,
            embed_dim=2 * glove_dim,
            use_embedding=self.use_embedding,
            hidden_dim=self.hidden_dim,
            output_dim=self.n_classes_,
            bidirectional=self.bidirectional,
            device=self.device
        )

    def predict_proba(self, X):
        with torch.no_grad():
            X_prem, X_hyp = zip(*X)
            X_prem, prem_lengths = self._prepare_dataset(X_prem)
            X_hyp, hyp_lengths = self._prepare_dataset(X_hyp)
            preds = self.model((X_prem, X_hyp), (prem_lengths, hyp_lengths))
            preds = torch.softmax(preds, dim=1).cpu().numpy()
            return preds

In [10]:
def sentence_encoding_rnn_phi(t1, t2):
    """Map `t1` and `t2` to a pair of lits of leaf nodes."""
    if hasattr(t1, 'leaves'):
        return (t1.leaves(), t2.leaves())
    else:
        return (t1, t2)

In [11]:
def get_sentence_encoding_vocab(X, n_words=None):
    wc = Counter([w for pair in X for ex in pair for w in ex])
    wc = wc.most_common(n_words) if n_words else wc.items()
    vocab = {w for w, c in wc}
    vocab.add("$UNK")
    #vocab.add(glove_vec("UNK"), GLOVE, is_srl=True)
    return set(vocab)

In [12]:
import datetime

def fit_sentence_encoding_rnn(X, y, raw_x):
    print("Getting vocab")
    vocab = get_sentence_encoding_vocab(raw_x, n_words=10000)
    print("Done getting vocab")

    mod = TorchRNNSentenceEncoderClassifier(
        vocab, hidden_dim=50, max_iter=50, embed_dim=2 * glove_dim, use_embedding=False)
    print("Fitting model")
    mod.fit(X, y)
    return mod

In [13]:
addamod_train_reader = nli.AddamodTrainReader(DATA_HOME)
addamod_dev_reader = nli.AddamodDevReader(DATA_HOME)
subobj_train_reader = nli.SubObjTrainReader(DATA_HOME)
subobj_dev_reader = nli.SubObjDevReader(DATA_HOME)
breaking_reader = nli.BreakingSNLIReader(DATA_HOME)

In [14]:
import string
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
wt = WordTokenizer()

In [15]:
def glove_srl_phi(ex, np_func=np.sum):
    prem = build_arrays(ex.sentence1, ex.tags1, glove_lookup)
    hyp = build_arrays(ex.sentence2, ex.tags2, glove_lookup)
#     if type(prem) != torch.Tensor or type(hyp) != torch.Tensor:
#         return None
    
    return (prem, hyp)


def build_arrays(sentence, tags, lookup):
    s = wt.tokenize(sentence)
    if len(tags) != len(s):
        print(f"Discarding this sentence because of tag issues: {sentence}")
#         print(len(tags))
#         print(len(s))
#         print(tags)
#         print(s)
        combo = torch.tensor(np.concatenate(([utils.randvec(n=glove_dim) for t in tags],
                                             [utils.randvec(n=glove_dim) for t in tags]), axis=1),
                             dtype=torch.float32, requires_grad=False)
    else:
        combo = torch.cat((torch.stack([_glove_vec(w, lookup) if w in lookup else _glove_vec("$UNK", lookup) for w in s]),
                           torch.stack([_get_tag_vec(t, srl_lookup) for t in tags])), 1)
#     if torch.cuda.is_available():
# #         print("Putting embedding on cuda.")
#         return combo.cuda()
#     else:
    return combo
    

def _glove_vec(w, lookup):
    vec = torch.tensor(lookup.get(w, lookup.get("$UNK", utils.randvec(n=glove_dim))), dtype=torch.float32, requires_grad=False)
    return vec
    
    
def _get_tree_vecs(tree, lookup, np_func):
    if hasattr(tree, 'leaves'):
        allvecs = np.array([lookup[w] if w in lookup else lookup["$UNK"] for w in tree.leaves() ])
    else:
        allvecs = np.array([lookup[w] if w in lookup else lookup["$UNK"] for w in tree ])
    return allvecs


def _get_tag_vec(tag, tag_lookup):
    if tag in tag_lookup:
        vec = tag_lookup[tag]
    else:
        vec = torch.tensor(utils.randvec(n=glove_dim), dtype=torch.float32, requires_grad=True)
        tag_lookup[tag] = vec
    return vec

In [ ]:
#Use this cell for training on SNLI, using an adversarial reader as the validation set.

snli_267379 = nli.experiment(
    train_reader=nli.NLIReader("/home/brett_szalapski/cs224uSNLI/data/nlidata/preproc_copies/"
                               "preprocessed_snli_1.0_train.jsonl",
                               samp_percentage=None), 
    phi=glove_srl_phi,
    train_func=fit_sentence_encoding_rnn,
    assess_reader=nli.NLIReader("/home/brett_szalapski/cs224uSNLI/data/nlidata/preproc_copies/"
                                "preprocessed_snli_1.0_dev.jsonl",
                                samp_percentage=None), 
    random_state=42,
    vectorize=False)

In [25]:
import pickle
with open("srl_375800.obj", "wb") as fp:
    pickle.dump(snli_267379, fp)

In [16]:
with open("srl_267379.obj", "rb") as fp:
    snli_267379 = pickle.load(fp)

In [ ]:
#Use this cell for using an adversarial reader as the validation set.

soswap_results = nli.evaluation(
    train_reader=nli.NLIReader("/home/brett_szalapski/cs224uSNLI/data/nlidata/preproc_copies/"
                               "preprocessed_snli_1.0_train.jsonl",
                               samp_percentage=1), 
    mod=snli_267379['model'],
    phi=glove_srl_phi,
    assess_reader=nli.NLIAdversaryReader("/home/brett_szalapski/cs224uSNLI/data/nlidata/preproc_copies/"
                                         "preprocessed_sub_obj_swap(dev).jsonl",
                                         samp_percentage=1), 
    random_state=42,
    vectorize=False)

In [27]:
addamod_results = nli.evaluation(
    train_reader=nli.NLIReader("/home/brett_szalapski/cs224uSNLI/data/nlidata/preproc_copies/"
                               "preprocessed_snli_1.0_train.jsonl",
                               samp_percentage=1), 
    mod=snli_267379['model'],
    phi=glove_srl_phi,
    assess_reader=nli.NLIAdversaryReader("/home/brett_szalapski/cs224uSNLI/data/nlidata/preproc_copies/"
                                         "preprocessed_add_amod(dev).jsonl",
                                         samp_percentage=1), 
    random_state=42,
    vectorize=False)

Discarding this sentence because of tag issues: A man with a Mohawk and a shirt saying"-ependent "faces the camera
Discarding this sentence because of tag issues: A man with a Mohawk and a shirt saying"-ependent "faces the camera
Discarding this sentence because of tag issues: A man with a Mohawk and a shirt saying"-ependent "faces the camera
Discarding this sentence because of tag issues: The little baseball player wearing the `` Irish '' uniform prepares to throw the ball to his teammate as the umpire watches closely .
Discarding this sentence because of tag issues: The baseball little player wearing the `` Irish '' uniform prepares to throw the ball to his teammate as the umpire watches closely .
Discarding this sentence because of tag issues: Many multiple people take pictures as a blond woman wearing a cover-up only costume holds a sign proclaiming `` Photographers Not Predators '' on a pier .
Discarding this sentence because of tag issues: Many people take multiple pictures as a 

In [25]:
breaking_results = nli.evaluation(
    train_reader=nli.NLIReader("/home/brett_szalapski/cs224uSNLI/data/nlidata/preproc_copies/"
                               "preprocessed_snli_1.0_train.jsonl",
                               samp_percentage=1), 
    mod=snli_267379['model'],
    phi=glove_srl_phi,
    assess_reader=nli.NLIReader("/home/brett_szalapski/cs224uSNLI/data/nlidata/preproc_copies/"
                                         "preprocessed_dataset.jsonl",
                                         samp_percentage=1), 
    random_state=42,
    vectorize=False)

Discarding this sentence because of tag issues: A man with a Mohawk and a shirt saying"-ependent "faces the camera
Discarding this sentence because of tag issues: A man with a Mohawk and a shirt saying"-ependent "faces the camera
Discarding this sentence because of tag issues: A man with a Mohawk and a shirt saying"-ependent "faces the camera


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:525: RuntimeWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  self.num_layers, self.dropout, self.training, self.bidirectional)


               precision    recall  f1-score   support

contradiction      0.857     0.129     0.224      6991
   entailment      0.098     0.005     0.010       964
      neutral      0.005     0.822     0.011        45

    micro avg      0.118     0.118     0.118      8000
    macro avg      0.320     0.319     0.081      8000
 weighted avg      0.761     0.118     0.197      8000



In [17]:
snli_test_results = nli.evaluation(
    train_reader=nli.NLIReader("/home/brett_szalapski/cs224uSNLI/data/nlidata/preproc_copies/"
                               "preprocessed_snli_1.0_train.jsonl",
                               samp_percentage=0.01), 
    mod=snli_267379['model'],
    phi=glove_srl_phi,
    assess_reader=nli.NLIReader("/home/brett_szalapski/cs224uSNLI/data/nlidata/preproc_copies/"
                                "preprocessed_snli_1.0_test.jsonl",
                                samp_percentage=None), 
    random_state=42,
    vectorize=False)

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:525: RuntimeWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  self.num_layers, self.dropout, self.training, self.bidirectional)


               precision    recall  f1-score   support

contradiction      0.314     0.005     0.010      3229
   entailment      0.353     0.433     0.389      3361
      neutral      0.334     0.587     0.426      3210

    micro avg      0.342     0.342     0.342      9800
    macro avg      0.334     0.341     0.275      9800
 weighted avg      0.334     0.342     0.276      9800

